## Crowdfunding Analysis - Final Project

#### Dataset: https://www.kaggle.com/yashkantharia/kickstarter-campaigns-dataset-20


### Importing dependecies and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # plotting

df = pd.read_csv('data.csv')

In [2]:
# Describing the Data
# df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217245 entries, 0 to 217244
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   index          217245 non-null  int64  
 1   id             217245 non-null  int64  
 2   name           217245 non-null  object 
 3   currency       217245 non-null  object 
 4   launched_at    217245 non-null  object 
 5   backers_count  217245 non-null  int64  
 6   blurb          217245 non-null  object 
 7   country        217245 non-null  object 
 8   deadline       217245 non-null  object 
 9   slug           217245 non-null  object 
 10  status         217245 non-null  object 
 11  usd_pledged    217245 non-null  float64
 12  sub_category   217245 non-null  object 
 13  main_category  217245 non-null  object 
 14  creator_id     217245 non-null  int64  
 15  blurb_length   217245 non-null  int64  
 16  goal_usd       217245 non-null  float64
 17  city           217245 non-nul

### Data Cleaning

* Duplicates
* Removing useless columns

In [3]:
# creator_id, index, creator_id are unique / serializing values
ul_cols = ['id', 'creator_id', 'index']
df = df.drop(columns = ul_cols, axis=1)

In [5]:
# Removing Duplicates
print("# of Duplicates: ", df.duplicated().sum())
df.drop_duplicates(inplace=True, ignore_index=True)

# Length after duplicate removal
print("Length of dataset after removal of duplicates is ", len(df))

duplicates:  19527
length of dataset after removal of duplicates is  197718


### Importing models
* Logistic Regrssion
* Random Forest
